In [3]:
#https://discuss.pytorch.org/t/convolution-that-only-take-channel-wise-summation/21240
#https://auro-227.medium.com/writing-a-custom-layer-in-pytorch-14ab6ac94b77
#https://towardsdatascience.com/how-to-build-your-own-pytorch-neural-network-layer-from-scratch-842144d623f6
#https://colab.research.google.com/github/d2l-ai/d2l-en-colab/blob/master/chapter_deep-learning-computation/custom-layer.ipynb
#https://discuss.pytorch.org/t/custom-convolution-dot-product/14992/12
#https://discuss.pytorch.org/t/efficiently-slicing-tensor-like-a-convolution/44840/2
#https://discuss.pytorch.org/t/custom-a-new-convolution-layer-in-cnn/43682/14
#https://towardsdatascience.com/convolutional-layer-hacking-with-python-and-numpy-e5f64812ca0c
#https://github.com/pskugit/custom-conv2d
#https://discuss.pytorch.org/t/custom-nn-conv2d/62068
#https://stackoverflow.com/questions/59149785/custom-conv2d-operation-pytorch

In [18]:
import math
import torch
import torch.nn as nn
import numpy as np
import copy

In [24]:
class MyLinearLayer(nn.Module):
    """ Custom Linear layer but mimics a standard linear layer """
    def __init__(self, size_in, size_out):
        super().__init__()
        self.size_in, self.size_out = size_in, size_out
        weights = torch.Tensor(size_out, size_in)
        self.weights = nn.Parameter(weights)  # nn.Parameter is a Tensor that's a module parameter.
        bias = torch.Tensor(size_out)
        self.bias = nn.Parameter(bias)

        # initialize weights and biases
        nn.init.kaiming_uniform_(self.weights, a=math.sqrt(5)) # weight init
        fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weights)
        bound = 1 / math.sqrt(fan_in)
        
        #Checar uniform aqui
        nn.init.uniform_(self.bias, -bound, bound)  # bias init

    def forward(self, x):
        w_times_x= torch.mm(x, self.weights.t())
        return torch.add(w_times_x, self.bias)  # w times x + b


class BasicModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(1, 128, 3)
        self.linear = nn.Linear(256, 2)
        #self.linear = MyLinearLayer(256, 2)

    def forward(self, x):
        x = self. conv(x)
        x = x.view(-1, 256)
        return self.linear(x)


torch.manual_seed(0)  #  for repeatable results
basic_model = BasicModel()
basic_model2 = BasicModel()
inp = np.array([[[[1,2,3,4],  # batch(=1) x channels(=1) x height x width
                  [1,2,3,4],
                  [1,2,3,4]]]])
x = torch.tensor(inp, dtype=torch.float)
print('Forward computation thru model:', basic_model(copy.deepcopy(x)))

Forward computation thru model: tensor([[ 1.2722, -0.6224]], grad_fn=<AddmmBackward>)
